In [ ]:
import csv
import time
import os
import subprocess
import psutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization


class CustomCallback(keras.callbacks.Callback):
    def __init__(self):
        super(CustomCallback, self).__init__()
        self.training_time = None
        self.average_memory_utilization = None
        self.average_gpu_utilization = None
        
    def on_train_begin(self, logs={}):
        self.start_time = time.time()
        self.memory_utilizations = []
        self.gpu_utilizations = []
        
    def on_train_end(self, logs={}):
        self.training_time = time.time() - self.start_time
        self.average_memory_utilization = np.mean(self.memory_utilizations)
        self.average_gpu_utilization = np.mean(self.gpu_utilizations)
        print('\nTotal training time: {} seconds'.format(self.training_time))
        print('\nAverage memory utilization during training so far: {}'.format(self.average_memory_utilization))
        print('\nAverage GPU utilization during training so far: {}'.format(self.average_gpu_utilization))

    def on_epoch_end(self, epoch, logs={}):
        memory_utilization = psutil.cpu_percent()
        gpu_utilization_process = subprocess.Popen(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader'], stdout=subprocess.PIPE)
        gpu_utilization = int(gpu_utilization_process.stdout.readline().strip().decode('utf-8').split()[0])
        self.memory_utilizations.append(memory_utilization)
        self.gpu_utilizations.append(gpu_utilization)

        
def Classify(batch_size, epochs):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    callback = CustomCallback()
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epoch, validation_data=(x_test, y_test), callbacks=[callback], verbose=1)
    
    test_loss, test_acc = model.evaluate(x_test, y_test)
    
    with open('cifar10_deep_classification.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow([batch_size, epochs, callback.average_memory_utilization, callback.average_gpu_utilization, callback.training_time, test_loss, test_acc])

        
for batch in range(100, 801, 10):
    for epoch in range(10, 41, 10):
        Classify(batch, epoch)


2023-02-12 07:35:16.325594: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 07:35:16.344449: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 07:35:16.344657: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 07:35:16.345314: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/10


2023-02-12 07:35:18.836535: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8400
2023-02-12 07:35:19.280932: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-12 07:35:19.290055: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-12 07:35:19.290079: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-02-12 07:35:19.290509: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-12 07:35:19.290563: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


500/500 [==============================] - 8s 13ms/step - loss: 1.6555 - accuracy: 0.4341 - val_loss: 1.5979 - val_accuracy: 0.4309
Epoch 2/10
500/500 [==============================] - 6s 12ms/step - loss: 1.0856 - accuracy: 0.6099 - val_loss: 0.9868 - val_accuracy: 0.6566
Epoch 3/10
500/500 [==============================] - 6s 12ms/step - loss: 0.9036 - accuracy: 0.6818 - val_loss: 0.8778 - val_accuracy: 0.6948
Epoch 4/10
500/500 [==============================] - 6s 12ms/step - loss: 0.7990 - accuracy: 0.7209 - val_loss: 0.7386 - val_accuracy: 0.7420
Epoch 5/10
500/500 [==============================] - 6s 12ms/step - loss: 0.7289 - accuracy: 0.7461 - val_loss: 0.7445 - val_accuracy: 0.7480
Epoch 6/10
500/500 [==============================] - 6s 12ms/step - loss: 0.6662 - accuracy: 0.7694 - val_loss: 0.6047 - val_accuracy: 0.7896
Epoch 7/10
500/500 [==============================] - 6s 12ms/step - loss: 0.6132 - accuracy: 0.7901 - val_loss: 0.6013 - val_accuracy: 0.7926
Epoch 8/10